In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Rebalancing with a model portfolio 

This notebook shows how to you can automatically generate transactions to rebalance a transaction portfolio with a model portfolio

Attributes
----------
transactions portfolios
reference portfolios
"""

toggle_code("Hide docstring")

# 1. Introduction

This notebook shows how to you can setup a rebalancer to rebalance a transaction porfolio against a reference portfolio. Section 1 of this notebook demonstrates the data setup. Skip to Section 2 if you want to see the rebalancer in action. In Section 2 we rebalance an equity portfolio against a model equity portfolio which has set weightings for 9 FTSE equities. 

# 2. Setup Python environment

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from datetime import datetime, timedelta
from flatten_json import flatten

import os
import pandas as pd
import numpy as np
import pytz
import time
import json

# Set pandas dataframe display formatting
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# define LUSID APIs which we'll use below
quotes_api = api_factory.build(lusid.QuotesApi)
portfolio_api = api_factory.build(lusid.PortfoliosApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
aggregation_api = api_factory.build(lusid.AggregationApi)
reference_portfolio_api = api_factory.build(lusid.ReferencePortfolioApi)
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)
transaction_portfolio_api = api_factory.build(lusid.TransactionPortfoliosApi)

In [4]:
# define date variables

start_q1 = "2020-01-01"
start_q2 = "2020-04-01"

# 3. Data load 


<b>PLEASE NOTE: The following section loads all the data we'll use in Section 2. You can skip Section 1 if you want to go straight to the rebalancing activities.</b>

## 3.1 Load CSV files of transaction and constituent data

In [5]:
# Load csv file of transactions

transaction_portfolio_data_csv = r"data/rebalancer/transaction_portfolio_cash.csv"
transaction_portfolio_df = pd.read_csv(transaction_portfolio_data_csv)
transaction_portfolio_df

,fund_code,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,txn_instrument_id,currency
0,equityPortfolio,cash_1,FundsIn,01/01/2020,01/01/2020,10000000,1,10000000,GBP,GBP


In [6]:
# Load CSV file of constituents
constituent_file_csv = r"data/rebalancer/equity_constituents.csv"
constituent_df = pd.read_csv(constituent_file_csv)
constituent_df

,ticker,sedol,instrument_type,instrument_id,name,weighting
0,BT.A,SEDOL1,equity,EQ_1234,BT GROUP PLC,0.05
1,STAN,SEDOL2,equity,EQ_1235,STANDARD CHARTERED PLC,0.07
2,SBRY,SEDOL3,equity,EQ_1236,J SAINSBURY PLC,0.12
3,BARC,SEDOL4,equity,EQ_1237,BARCLAYS PLC,0.09
4,BP,SEDOL5,equity,EQ_1238,BP PLC,0.04
5,GSK,SEDOL6,equity,EQ_1239,GLAXOSMITHKLINE PLC,0.11
6,BRBY,SEDOL7,equity,EQ_1240,BURBERRY GROUP PLC,0.15
7,OCDO,SEDOL8,equity,EQ_1241,OCADO GROUP PLC,0.22
8,NXT,SEDOL9,equity,EQ_1242,NEXT PLC,0.15


## 3.2 Create transaction portfolio

In [7]:
scope = "iborScope"
transaction_portfolio_code = "equityPortfolio"
reference_portfolio_code = "equityPortfolioModel"

In [8]:
mapping_required = {
    "display_name": "fund_code",
    "code": "fund_code",
    "base_currency": "currency",
}


mapping_optional = {"created": "$2000-01-01"}

# Use the load_from_data_frame method from LUSID's Python cocoon package to upload the portfolio

response = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transaction_portfolio_df,
    property_columns=[],
    mapping_required=mapping_required,
    mapping_optional=mapping_optional,
    file_type="portfolios",
)

## 3.3 Create reference portfolio

In [9]:
mapping_required = {
    "display_name": "$" + reference_portfolio_code,
    "code": "$" + reference_portfolio_code,
    "base_currency": "currency",
}


mapping_optional = {"created": "$2000-01-01"}

# Use the load_from_data_frame method from LUSID's Python cocoon package to upload the portfolio

response = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transaction_portfolio_df,
    property_columns=[],
    mapping_required=mapping_required,
    mapping_optional=mapping_optional,
    file_type="reference_portfolio",
)

## 3.4 Create instrument master

In [10]:
# Create dictionaries of mappings

mapping_required = {
    "name": "name",
}

# This time, we also need to tell LUSID about our unique identifiers
# All instruments in LUSID need a unique identifier

identifiers = {
    "ClientInternal": "instrument_id",
    "Ticker": "ticker",
    "Sedol": "sedol",
}

properties = ["instrument_type"]

response = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=constituent_df,
    mapping_required=mapping_required,
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=identifiers,
    property_columns=properties,
)

## 3.5 Upload constituents

In [11]:
# Initialise a list to hold our constituents
constituents = []

# Iterate over instrument unvierse to add each constituent to our list
for index, row in constituent_df.iterrows():
    constituents.append(
        models.ReferencePortfolioConstituentRequest(
            instrument_identifiers={
                "Instrument/default/ClientInternal": row["instrument_id"]
            },
            weight=row["weighting"],
            currency="GBP",
        )
    )

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_q1,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=4,
    constituents=constituents,
)

# Call LUSID to upsert our constituents into our reference portfolio
response = reference_portfolio_api.upsert_reference_portfolio_constituents(
    scope=scope,
    code=reference_portfolio_code,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print("Constituents Upserted")

Constituents Upserted


## 3.6 Upload £1m cash into the transaction portfolio

In [12]:
mapping = {
    "transactions": {
        "identifier_mapping": {"Currency": "currency"},
        "required": {
            "code": "fund_code",
            "transaction_id": "txn_id",
            "type": "txn_type",
            "transaction_price.price": "txn_price",
            "transaction_price.type": "$Price",
            "total_consideration.amount": "txn_consideration",
            "units": "txn_units",
            "transaction_date": "txn_settle_date",
            "total_consideration.currency": "currency",
            "settlement_date": "txn_settle_date",
        },
    }
}

In [13]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transaction_portfolio_df,
    mapping_required=mapping["transactions"]["required"],
    mapping_optional={},
    file_type="transactions",
    identifier_mapping=mapping["transactions"]["identifier_mapping"],
    property_columns=[],
    properties_scope=scope,
)

## 3.7 Upload quotes

In [14]:
start_date = datetime(year=2020, month=1, day=1)
num_of_days = 100
days = [
    (
        start_date + timedelta(days=x)
    ).strftime("%Y-%m-%d") for x in range(num_of_days)
]

In [15]:
# define lambda function to get instrument information from lusid
get_instrument = lambda instrument_id: instruments_api.get_instrument(
    identifier_type="ClientInternal", 
    identifier=instrument_id,
)

# Generate mapping from Names to Luids in order to create price movement based on the industry of the equity
luids = {instrument.name: instrument.lusid_instrument_id for instrument in map(get_instrument, constituent_df["instrument_id"])}

quotes_format_str = "quotes_request_{id}_{date}"
    
def create_quotes(name: str, growth: float) -> dict:
    """
    Generates upsert quote request object for a given equity name at a given growth rate
    """
    prices = [10 + (x * growth) for x in range(len(days))]
    daily_prices = zip(days, prices)

    return {
        quotes_format_str.format(id=luids[name], date=date.replace("-", "")): models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=luids[name],
                    instrument_id_type="LusidInstrumentId",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=date,
            ),
            metric_value=models.MetricValue(
                value=price, 
                unit="GBP",
            ),
        ) for date, price in daily_prices
    }

Oil and gass outperform portfolio average as BP PLC market price increase over time.

Banking underperfoms when compared to portfolio average as Barclays and Standard Chartered PLC market price fall over time.

In [16]:
# quotes to be upserted
quotes_for_upsert = {}

for name, luid in luids.items():
    if name not in ["BARCLAYS PLC", "STANDARD CHARTERED PLC", "BP PLC"]:
        quotes_for_upsert.update(create_quotes(name=name, growth=0.05))

# Create quotes for Barclays and Standard bank
barclays_quotes = create_quotes(name="BARCLAYS PLC", growth=-0.025)
standard_quotes = create_quotes(name="STANDARD CHARTERED PLC", growth=-0.031)

# Create quotes for BP
bp_quotes = create_quotes(name="BP PLC", growth=0.083)

quotes_for_upsert.update(barclays_quotes)
quotes_for_upsert.update(standard_quotes)
quotes_for_upsert.update(bp_quotes)

In [17]:
upsert_quotes_response = quotes_api.upsert_quotes(
    scope=scope,
    request_body=quotes_for_upsert,
)

## 3.8 Create recipe

In [18]:
try:

    inline_recipe = models.ConfigurationRecipe(
        scope=scope,
        code="rebalancer_recipe",
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.LusidInstrumentId.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                )
            ],
            suppliers=models.MarketContextSuppliers(
                commodity="Lusid", credit="Lusid", equity="Lusid", fx="Lusid", rates="Lusid"
            ),
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="LusidInstrumentId",
                default_scope=scope,
            ),
        ),
    )

    upsert_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(configuration_recipe=inline_recipe)
    )

    print("Quotes upserted successfully")

except lusid.ApiException as e:
    error = json.loads(e.body)
    print(json.dumps(error, indent=4))

Quotes upserted successfully


## 3.9 Define the rebalancing functions 

In [19]:
def portfolio_rebalancer(txn_port: str, reference_port: str, date: str) -> None:

    title = f"Running rebalancer of {txn_port} against model portfolio {reference_port}"
    print(f"\n{'-' * len(title)}\n{title}\n{'-' * len(title)}")

    # Calculate the value of the portfolio before rebalance
    cash_to_invest_df = run_agg(date, txn_port)
    total_portfolio_value = cash_to_invest_df["pv"].sum()
    print(f"\nThe value of the portfolio before rebalance on {date} is {total_portfolio_value}\n")

    # Calculate the the new target PV for each position
    constituents = get_reference_port_constituents(reference_port, scope)
    targets = constituents.loc[:, ["instrument_uid", "weight"]]
    targets["total_portfolio_value"] = total_portfolio_value

    targets_formatted = targets.set_index("instrument_uid")
    targets_formatted["target_pv"] = targets_formatted["total_portfolio_value"].multiply(targets_formatted["weight"])

    # Print 
    for index, row in targets_formatted.iterrows():
        print(f"Creating new PV target of {row['target_pv'].round(2)} for instrument {index}")

    # Get quotes for adjustment date
    quotes_df = get_quotes_df(date)
    target_with_quotes = pd.concat([targets_formatted, quotes_df], axis=1)

    # Calculate new target units for adjustment date
    target_with_quotes["target_units"] = (
        (target_with_quotes["target_pv"] / target_with_quotes["price"])
    )#.round(2)

    # Post adjustment trades
    current_holdings = get_current_holdings(txn_port, date)[
        ["units", "instrument_uid"]
    ].set_index("instrument_uid")
    new_trade_requirements = pd.concat(
        [target_with_quotes, current_holdings], axis=1, sort=False
    ).fillna(0)
    new_trade_requirements["adjustment_units"] = (
        new_trade_requirements["target_units"] - new_trade_requirements["units"]
    )
    new_trade_requirements = new_trade_requirements[
        new_trade_requirements.index.str.contains("LUID")
    ]

    txn_id_ts = time.time()

    new_trade_requirements["transaction_id"] = ["trd_" + str(txn_id_ts) + str(i) for i in range(len(new_trade_requirements))]
    new_trade_requirements["type"] = np.where(new_trade_requirements["adjustment_units"] > 0, "Buy", "Sell")
    new_trade_requirements["adjustment_units"] = new_trade_requirements["adjustment_units"].abs()
    new_trade_requirements["total_cost"] = new_trade_requirements["adjustment_units"] * new_trade_requirements["price"]
    new_trade_requirements = new_trade_requirements.loc[new_trade_requirements.adjustment_units > 0, :]

    print("\nUpserting new trades to adjust portfolio")
    upsert_trades(new_trade_requirements, scope, txn_port, date)
    print("\nRebalancer is FINISHED")

In [20]:
def run_agg(date: str, portfolio_code: str) -> pd.DataFrame:
    """
    Aggregates metric values (Pv, Units, Cost and Price) up untill specified date
    """
    # Create the valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code="rebalancer_recipe"),
        metrics=[
            models.AggregateSpec(key="Holding/default/SubHoldingKey", op="Value"),
            models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
            models.AggregateSpec(key="Holding/default/PV", op="Sum"),
            models.AggregateSpec(key="Holding/default/Units", op="Sum"),
            models.AggregateSpec(key="Instrument/default/Name", op="Value"),
            models.AggregateSpec(key="Quotes/Price", op="Value"),
            models.AggregateSpec(
                key="Instrument/default/LusidInstrumentId", op="Value"
            ),
        ],
        group_by=["Instrument/default/LusidInstrumentId"],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
    )

    try:

        # Perform a valuation
        valuation = aggregation_api.get_valuation(
            valuation_request=valuation_request
        )

        response_result = pd.DataFrame(data=[
            {
                "date": date,
                "portfolio": portfolio_code,
                "luid": item["Instrument/default/LusidInstrumentId"],
                "pv": item["Sum(Holding/default/PV)"],
                "units": item["Sum(Holding/default/Units)"],
                "price": item["Quotes/Price"],
            } for item in valuation.data
        ])

    except lusid.ApiException as e:
        
        # Print out error
        print(json.dumps(e.body, indent=4))
        
        response_result = pd.DataFrame()
    
    return response_result

In [21]:
def get_reference_port_constituents(reference_port: str, reference_scope: str) -> pd.DataFrame:
    """
    Returns Portfolion Constituents from existing reference portfolio
    """
    response = reference_portfolio_api.get_reference_portfolio_constituents(
        scope=reference_scope,
        code=reference_port,
        property_keys=["Instrument/default/Name"],
    )

    return pd.DataFrame([flatten(item.to_dict()) for item in response.constituents])

In [22]:
def get_quotes_df(date: str) -> pd.DataFrame:

    filter_date = date + "T00:00:00.0000000+00:00"

    quotes_response = quotes_api.list_quotes_for_scope(
        scope=scope, filter=f"quoteId.effectiveAt eq {filter_date}"
    )

    quotes_df = lusid_response_to_data_frame(quotes_response).rename(
        columns={
            "quote_id.quote_series_id.instrument_id": "luid",
            "metric_value.value": "price",
            "quote_id.effective_at": "quote_date",
        }
    )

    quotes_df = quotes_df[["luid", "price", "quote_date"]]

    quotes_df = quotes_df.set_index("luid")

    return quotes_df

In [23]:
def get_current_holdings(portfolio_code: str, date: str) -> pd.DataFrame:
    """
    Return current holdings as a DataFrame object
    """
    transaction_response = transaction_portfolio_api.get_holdings(
        scope=scope, 
        code=portfolio_code, 
        effective_at=date,
    )
    return lusid_response_to_data_frame(lusid_response=transaction_response)

In [24]:
def upsert_trades(analyst_transactions: pd.DataFrame, port_scope: str, portfolio_code: str, date: str) -> None:
    """
    Upserts transaction from a given transaction DataFrame `analyst_transaction` using LUSID's `TransactionRequest` model
    """
    # Initialise a list to hold our transactions
    batch_transaction_requests = []

    # Iterate over the transactions for each portfolio
    batch_transaction_requests = [
        models.TransactionRequest(
            transaction_id=transaction["transaction_id"],
            type=transaction["type"],
            instrument_identifiers={
                "Instrument/default/LusidInstrumentId": index,
            },
            transaction_date=date,
            settlement_date=date,
            units=transaction["adjustment_units"],
            transaction_price=models.TransactionPrice(
                price=transaction["price"], 
                type="Price",
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=transaction["total_cost"], 
                currency="GBP",
            ),
            source="Client",
            transaction_currency="GBP",
            properties={},
        ) for index, transaction in analyst_transactions.iterrows()
    ]

    # Call LUSID to upsert our transactions
    transaction_portfolio_api.upsert_transactions(
        scope=port_scope,
        code=portfolio_code,
        transaction_request=batch_transaction_requests,
    )

In [25]:
def formatted_mv_df(portfolio_code: str, date: str) -> pd.DataFrame:
    """
    Formats Market Value data for a given date
    """
    
    rebalanced_port = run_agg(date, portfolio_code)

    # Calcultate PV and weighting
    rebalanced_port["total_mv"] = rebalanced_port["pv"].sum()
    rebalanced_port["weighting"] = rebalanced_port["pv"].divide(rebalanced_port["total_mv"]) * 100

    # Get name from lusid for each equity
    rebalanced_port["name"] = rebalanced_port["luid"].apply(
        lambda luid: instruments_api.get_instrument(
            identifier_type="LusidInstrumentId", 
            identifier=luid,
        ).name
    )

    return rebalanced_port.loc[rebalanced_port.units.round(2) > 0, :].reset_index(drop=True)

# 4. Run a rebalancer

In this section we rebalance an equity portfolio against a model equity portfolio which has set weightings in 9 FTSE 100 equities.

## 4.1 Check transaction portfolio before rebalance - 10m in GBP

* The portfolio has GBP cash only 

In [26]:
formatted_mv_df(transaction_portfolio_code, start_q1)

,date,portfolio,luid,pv,units,price,total_mv,weighting,name
0,2020-01-01,equityPortfolio,CCY_GBP,"10,000,000.00","10,000,000.00",None,"10,000,000.00",100.00,GBP


## 4.2 Check the constituents of the reference portfolio

* We see 9 equities with various weightings between 4% and 22%

In [27]:
constituent_response = reference_portfolio_api.get_reference_portfolio_constituents(
    scope=scope,
    code=reference_portfolio_code,
    effective_at=start_q1,
    property_keys=["Instrument/default/Name"],
)

constituents = pd.DataFrame(
    [flatten(item.to_dict()) for item in constituent_response.constituents]
)
constituents[
    [
        "instrument_identifiers_Instrument/default/ClientInternal",
        "properties_Instrument/default/Name_value_label_value",
        "weight",
    ]
]

,instrument_identifiers_Instrument/default/ClientInternal,properties_Instrument/default/Name_value_label_value,weight
0,EQ_1234,BT GROUP PLC,0.05
1,EQ_1235,STANDARD CHARTERED PLC,0.07
2,EQ_1236,J SAINSBURY PLC,0.12
3,EQ_1237,BARCLAYS PLC,0.09
4,EQ_1238,BP PLC,0.04
5,EQ_1239,GLAXOSMITHKLINE PLC,0.11
6,EQ_1240,BURBERRY GROUP PLC,0.15
7,EQ_1241,OCADO GROUP PLC,0.22
8,EQ_1242,NEXT PLC,0.15


## 4.3 Run the rebalancer

* The rebalancer posts transactions to allocate cash across the 9 equities

In [28]:
portfolio_rebalancer(transaction_portfolio_code, reference_portfolio_code, start_q1)


----------------------------------------------------------------------------------
Running rebalancer of equityPortfolio against model portfolio equityPortfolioModel
----------------------------------------------------------------------------------

The value of the portfolio before rebalance on 2020-01-01 is 10000000.0

Creating new PV target of 500000.0 for instrument LUID_00003DVW
Creating new PV target of 700000.0 for instrument LUID_00003DVT
Creating new PV target of 1200000.0 for instrument LUID_00003DVU
Creating new PV target of 900000.0 for instrument LUID_00003DVV
Creating new PV target of 400000.0 for instrument LUID_00003DVZ
Creating new PV target of 1100000.0 for instrument LUID_00003DVR
Creating new PV target of 1500000.0 for instrument LUID_00003DVY
Creating new PV target of 2200000.0 for instrument LUID_00003DVS
Creating new PV target of 1500000.0 for instrument LUID_00003DVX

Upserting new trades to adjust portfolio

Rebalancer is FINISHED


## 4.4 Check transaction portfolio after rebalance

* The transaction portfolio now has holdings in the 9 equities

In [29]:
formatted_mv_df(transaction_portfolio_code, start_q1)

,date,portfolio,luid,pv,units,price,total_mv,weighting,name
0,2020-01-01,equityPortfolio,LUID_00003DVW,"500,000.00","50,000.00",10.00,"10,000,000.00",5.00,BT GROUP PLC
1,2020-01-01,equityPortfolio,LUID_00003DVT,"700,000.00","70,000.00",10.00,"10,000,000.00",7.00,STANDARD CHARTERED PLC
2,2020-01-01,equityPortfolio,LUID_00003DVU,"1,200,000.00","120,000.00",10.00,"10,000,000.00",12.00,J SAINSBURY PLC
3,2020-01-01,equityPortfolio,LUID_00003DVV,"900,000.00","90,000.00",10.00,"10,000,000.00",9.00,BARCLAYS PLC
4,2020-01-01,equityPortfolio,LUID_00003DVZ,"400,000.00","40,000.00",10.00,"10,000,000.00",4.00,BP PLC
5,2020-01-01,equityPortfolio,LUID_00003DVR,"1,100,000.00","110,000.00",10.00,"10,000,000.00",11.00,GLAXOSMITHKLINE PLC
6,2020-01-01,equityPortfolio,LUID_00003DVY,"1,500,000.00","150,000.00",10.00,"10,000,000.00",15.00,BURBERRY GROUP PLC
7,2020-01-01,equityPortfolio,LUID_00003DVS,"2,200,000.00","220,000.00",10.00,"10,000,000.00",22.00,OCADO GROUP PLC
8,2020-01-01,equityPortfolio,LUID_00003DVX,"1,500,000.00","150,000.00",10.00,"10,000,000.00",15.00,NEXT PLC


## 4.5 Run rebalancer again at start of next quarter

In [30]:
portfolio_rebalancer(transaction_portfolio_code, reference_portfolio_code, start_q2)


----------------------------------------------------------------------------------
Running rebalancer of equityPortfolio against model portfolio equityPortfolioModel
----------------------------------------------------------------------------------

The value of the portfolio before rebalance on 2020-04-01 is 13539900.0

Creating new PV target of 676995.0 for instrument LUID_00003DVW
Creating new PV target of 947793.0 for instrument LUID_00003DVT
Creating new PV target of 1624788.0 for instrument LUID_00003DVU
Creating new PV target of 1218591.0 for instrument LUID_00003DVV
Creating new PV target of 541596.0 for instrument LUID_00003DVZ
Creating new PV target of 1489389.0 for instrument LUID_00003DVR
Creating new PV target of 2030985.0 for instrument LUID_00003DVY
Creating new PV target of 2978778.0 for instrument LUID_00003DVS
Creating new PV target of 2030985.0 for instrument LUID_00003DVX

Upserting new trades to adjust portfolio

Rebalancer is FINISHED


## 4.6 Check adjustments created from rebalance

In [31]:
formatted_mv_df(transaction_portfolio_code, start_q2)

,date,portfolio,luid,pv,units,price,total_mv,weighting,name
0,2020-04-01,equityPortfolio,LUID_00003DVW,"676,995.00","46,528.87",14.55,"13,539,900.00",5.00,BT GROUP PLC
1,2020-04-01,equityPortfolio,LUID_00003DVT,"947,793.00","132,022.98",7.18,"13,539,900.00",7.00,STANDARD CHARTERED PLC
2,2020-04-01,equityPortfolio,LUID_00003DVU,"1,624,788.00","111,669.28",14.55,"13,539,900.00",12.00,J SAINSBURY PLC
3,2020-04-01,equityPortfolio,LUID_00003DVV,"1,218,591.00","157,746.41",7.72,"13,539,900.00",9.00,BARCLAYS PLC
4,2020-04-01,equityPortfolio,LUID_00003DVZ,"541,596.00","30,854.90",17.55,"13,539,900.00",4.00,BP PLC
5,2020-04-01,equityPortfolio,LUID_00003DVR,"1,489,389.00","102,363.51",14.55,"13,539,900.00",11.00,GLAXOSMITHKLINE PLC
6,2020-04-01,equityPortfolio,LUID_00003DVY,"2,030,985.00","139,586.60",14.55,"13,539,900.00",15.00,BURBERRY GROUP PLC
7,2020-04-01,equityPortfolio,LUID_00003DVS,"2,978,778.00","204,727.01",14.55,"13,539,900.00",22.00,OCADO GROUP PLC
8,2020-04-01,equityPortfolio,LUID_00003DVX,"2,030,985.00","139,586.60",14.55,"13,539,900.00",15.00,NEXT PLC


# 5. Cleanup - cancel all transactions

In [32]:
try:

    delete_portfolio_response = portfolio_api.delete_portfolio(scope, transaction_portfolio_code)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])